In [239]:
import numpy as np
import glob
#import matplotlib.pyplot as plt
#%matplotlib inline
from skimage.io import imread
from skimage import img_as_float

In [85]:
DATASET_DIR = "/Volumes/CB_RESEARCH/adience_face/"
IMG_FOLDER = "aligned"

-----

In [32]:
# (0-2, 4-6, 8-13, 15-20, 25-32, 38-43, 48-53, 60-)

In [114]:
def _read_fold(filename, debug=False):
    ages = dict() # debug
    dd = dict() # map column names to indices
    classes = {
        "(0, 2)": 0, "(4, 6)": 1, "(8, 12)": 2, "(15, 20)": 3, "(25, 32)": 4, "(38, 43)": 5, "(48, 53)": 6, "(60, 100)": 7
    }
    delim = "\t"
    # -----
    X, y = [], []
    with open(filename) as f:
        header = f.readline().rstrip().split(delim)
        for k in range(0, len(header)):
            dd[ header[k] ] = k
        for line in f:
            line = line.rstrip().split(delim)
            #print line
            #print line[dd["user_id"]], line[dd["original_image"]], line[dd["face_id"]], line[dd["age"]]
            img_loc = "%s/%s/%s/landmark_aligned_face.%s.%s" % \
                (DATASET_DIR, IMG_FOLDER, line[dd["user_id"]], line[dd["face_id"]], line[dd["original_image"]])
            if line[dd["age"]] in classes:
                #print img_loc, classes[ line[dd["age"]] ]
                X.append(img_loc)
                y.append(classes[ line[dd["age"]] ])          
            if line[dd["age"]] not in ages:
                ages[ line[dd["age"]] ] = 0
            ages[ line[dd["age"]] ] += 1
    if debug:
        print ages
    return X, y

In [115]:
def get_fold(valid_fold_idx):
    assert 0 <= valid_fold_idx <= 3
    tot_folds = ["fold_0_data.txt", "fold_1_data.txt", "fold_2_data.txt", "fold_3_data.txt"]
    training_folds = [ tot_folds[i] for i in range(0, len(tot_folds)) if i != valid_fold_idx ]
    valid_fold = tot_folds[valid_fold_idx]
    training_X = []
    training_y = []
    for training_fold in training_folds:
        x, y = _read_fold("%s/%s" % (DATASET_DIR, training_fold))
        training_X += x
        training_y += y
    valid_X, valid_y = _read_fold("%s/%s" % (DATASET_DIR, valid_fold))
    return training_X, training_y, valid_X, valid_y

In [116]:
xt, yt, xv, yv = get_fold(0)

In [121]:
len(xt), len(xv)

(9340, 4377)

----

In [163]:
import os
os.environ["HDF5_DISABLE_VERSION_CHECK"] = "1"
import keras
from keras.preprocessing.image import ImageDataGenerator

Using Theano backend.


In [241]:
IMGEN = ImageDataGenerator(horizontal_flip=True)

In [242]:
def load_image_keras(filename, crop):
    img = imread(filename)
    img = img_as_float(img)
    img = np.asarray( [ img[...,0], img[...,1], img[...,2] ] ) # reshape
    for i in range(0, img.shape[0]):
        img[i, ...] = (img[i, ...] - np.mean(img[i, ...])) / np.std(img[i,...]) # zmuv
    for xb, _ in IMGEN.flow( np.asarray([img], dtype=img.dtype), np.asarray([0], dtype="int32")):
        if crop != None:
            img_size = xb[0].shape[-1]
            #print xb[0].shape
            #print img_size
            x_start = np.random.randint(0, img_size-crop+1)
            y_start = np.random.randint(0, img_size-crop+1)
            ret = xb[0][:, y_start:y_start+crop, x_start:x_start+crop]
            #print ret.shape
            return ret
        break
    return xb[0]

In [240]:
#plt.imshow( load_image_keras(xt[1], imgen, crop=600)[0], cmap="gray" )